In [29]:
# Input the complete matrix, not just lower triangular

coxeter_matrix = matrix([[1,3,2], [3,1,6],[2,6,1]])

dim_v = coxeter_matrix.rank()
# Vector space over the reals
v = VectorSpace(RR,dim_v)

In [30]:
coxeter_matrix

[1 3 2]
[3 1 6]
[2 6 1]

In [31]:
def bilinear(x,y):
    '''Computes the bilinear form of two roots'''
    temp = 0
    for i in range(dim_v):
        for j in range(dim_v):
            temp += x[i]*y[j]*(-cos(pi/coxeter_matrix[i,j])) 
    return temp

def s(i):
    '''Returns the ith simple root'''
    return v.basis()[i]

def simple_reflection(s,x):
    return x-2*bilinear(x,s)*s

def w_action(word_list,x):
    '''Input a word w in the generators as a list and a root \alpha. Returns w(\alpha)'''
    temp_x = x
    word_list.reverse()
    
    for s in word_list:
        temp_x = simple_reflection(s,temp_x)
    
    # reverse back is it reverses the word outside this function as well
    word_list.reverse()
    
    return temp_x

def inversion_set(w):
    '''Returns the left inversion set of an element w as a list'''
    inversionlist = [w[0]]

    for i in range(len(w)-1):        
        # note: w[0:i+1] does not include the endpoint i+1
        temp = w[0:i+1]
        inversionlist.append(w_action(temp,w[i+1]))
        
    return inversionlist

def elementary_roots():
    '''Returns the set of elementary roots'''
    elementary = []
    
    for i in range(dim_v):
        elementary.append(s(i))
    
    temp_n = 1
    counter = 0
    
    while temp_n < len(elementary):
        
        new_root_counter = 0
        
        temp_n = len(elementary)

        for root in elementary[counter:]:
            for i in range(dim_v):

                temp = w_action([s(i)],root)

                if temp[i] > root[i] and temp[i]-root[i] < 2 and temp not in elementary:
                    elementary.append(temp)
                    new_root_counter += 1
        
        counter = len(elementary) - new_root_counter - 1
                      
    return elementary

def elementary_inversion_set(w, elementary):
    '''returns the left elementary inversion set of an element, but compute the elementary roots first and input it, 
       to avoid recalculating it everytime.'''
    
    inver = inversion_set(w)
    
    e_inversion = [root for root in elementary if root in inver]
    
    return e_inversion

def left_length_increase(word, s):
    '''Tells you whether a word increases in length by multiplication by s on the left'''
    
    if word[0] == s:
        return False
    
    word_copy = word[:]
    word_copy.reverse()
    
    root = w_action(word_copy,s)
    
    if sum(root) >= 0:
        return True
    else:
        return False
    
def words_are_same_element(word1, word2):
    '''Given two reduced words, determine whether they are the same element 
    by checking if their inversion sets are the same'''
    
    # this only works because we assume words are reduced
    if len(word1) != len(word2):
        return False
  
    w1_inversion = inversion_set(word1)
    w2_inversion = inversion_set(word2)
        
    for root in w1_inversion:
        if root in w2_inversion:
            w2_inversion.remove(root)
    
    if len(w2_inversion) == 0:
        return True
    else:
        return False

def word_nice(word):
    '''prints word as a string of letters'''
    word_string = ''
    for vector in word:
        if vector == s(0):
            word_string += 's'
        elif vector == s(1):
            word_string += 't'
        elif vector == s(2):
            word_string += 'u'
        else:
            word_string += 'v'
    
    return word_string

def wordnice_to_vectorlist(word):
    '''Given a nice presented word returns a list of simple vectors representing the word for computation'''
    word_list = []
    
    for letter in word:
        if letter == 's':
            word_list.append(s(0))
        if letter == 't':
            word_list.append(s(1))
        if letter == 'u':
            word_list.append(s(2))
        if letter == 'v':
            word_list.append(s(3))
    
    return word_list


def elementary_inversion_lists_same(e1, e2):
    '''given two elementary inversion lists, check whether they are the same as sets'''
    if len(e1) != len(e2):
        return False
    
    e1_copy = e1[:]
    
    for root in e1:
        if root in e2:
            e1_copy.remove(root)
        else:
            return False
    
    if len(e1_copy) == 0:
        return True
    else:
        return False

In [32]:
elementary = elementary_roots()
elementary

[(1.00000000000000, 0.000000000000000, 0.000000000000000),
 (0.000000000000000, 1.00000000000000, 0.000000000000000),
 (0.000000000000000, 0.000000000000000, 1.00000000000000),
 (1.00000000000000, 1.00000000000000, 0.000000000000000),
 (0.000000000000000, 1.00000000000000, 1.00000000000000*sqrt(3)),
 (0.000000000000000, 1.00000000000000*sqrt(3), 1.00000000000000),
 (1.00000000000000, 1.00000000000000, 1.00000000000000*sqrt(3)),
 (0.000000000000000, 2.00000000000000, 1.00000000000000*sqrt(3)),
 (1.00000000000000*sqrt(3), 1.00000000000000*sqrt(3), 1.00000000000000),
 (0.000000000000000, 1.00000000000000*sqrt(3), 2.00000000000000),
 (1.00000000000000*sqrt(3), 1.00000000000000*sqrt(3), 2.00000000000000),
 (1.00000000000000*sqrt(3), 2.00000000000000*sqrt(3), 2.00000000000000)]

## Some code for dihedral root subsystems

In [4]:
import math

def dihedral_root_system(r1, r2):
    '''Returns the root system for the dihedral group generated by the reflections r1 and r2
       input is two reflections as a list
    '''
    
    # Get root associated to reflection 1
    middle_of_r1 = int(math.ceil(len(r1)/2))
    
    if middle_of_r1 - 2==0:
        root1 = w_action([r1[0]],r1[middle_of_r1 - 1])
    elif middle_of_r1 == 1:
        root1 = r1[0]
    else:
        root1 = w_action(r1[0:middle_of_r1 - 1],r1[middle_of_r1 - 1])
    
    # Get root associated to reflection 2
    middle_of_r2 = int(math.ceil(len(r2)/2))
    
    if middle_of_r2 - 2==0:
        root2 = w_action([r2[0]],r2[middle_of_r2 - 1])
    elif middle_of_r2 == 1:
        root2 = r2[0]
    else:
        root2 = w_action(r2[0:middle_of_r2 - 1],r2[middle_of_r2 - 1])
    
    roots = [root1, root2]
    reflections = [r1, r2]
    
    temp_n = 1
    counter = 0
    
    while temp_n < len(roots):
        
        new_root_counter = 0
        
        temp_n = len(roots)
        
        if temp_n > 200:
            print("Looks like this will be an infinite dihedral group. I'm going to stop!!")
            break
        
        for root in roots[counter:]:
            for re in reflections:
                new_root = w_action(re,root)
                
                if new_root not in roots:
                    roots.append(new_root)
                    new_root_counter += 1
        
        counter = len(roots) - new_root_counter - 1
        
    return roots

def positive_roots(subgroup):
    
    pos_roots = []
    
    for el in subgroup:
        if sum(el) > 0:
            pos_roots.append(el)
            
    return pos_roots

In [6]:
w_action([s(0),s(2),s(1)], s(2))

(1.00000000000000*sqrt(3), 1.00000000000000*sqrt(3), 2.00000000000000)

In [7]:
wordnice_to_vectorlist('stu')

[(1.00000000000000, 0.000000000000000, 0.000000000000000),
 (0.000000000000000, 1.00000000000000, 0.000000000000000),
 (0.000000000000000, 0.000000000000000, 1.00000000000000)]

## Construct the Brink-Howlett automata

In [33]:
def bh_automata():
    '''Returns the list of states, where each item is a list with first index a state
    and then following items in the list are tuples with a transition letter and target state'''

    # list of states and their transition information
    # as elementary inversion lists
    states_transitions = []
    
    # list of states only (as representative word)
    state_list = ['s', 't', 'u']
    
    counter = 0
    num_of_new_states = 1
    
    while num_of_new_states > 0:
        
        num_of_new_states = 0
        
        for state in state_list[counter:]:
            
            counter = len(state_list)-num_of_new_states
            
            state_vector = wordnice_to_vectorlist(state)
            
            state_elementary = elementary_inversion_set(state_vector, elementary)

            state_info = [state]
            
            for j in range(0,dim(v)):
                #print(' ')
                
                generator_word = word_nice([s(j)])
                #print('Word representing inversion set is: ' + state + ' generator is: ' + generator_word )
                
                if left_length_increase(state_vector,s(j)):

                    #print('Yes, ' + state + ' increases on the left by ' + word_nice([s(j)]))
                    
                    target_state = [s(j)] + state_vector
                    target_state_word = word_nice(target_state)
                    
                    #print('Word repping Target State: ' + target_state_word)
                    
                    target_state_elementary_inversion_set = elementary_inversion_set(target_state, elementary)
                    
                    # If target state elementary inversion set is not in the list of states, add to the list.
                    # if already in list, add appropriate transition
                    target_state_is_new = True
                    
                    for st in state_list:
                        st_elementary = elementary_inversion_set(wordnice_to_vectorlist(st), elementary)
                        
                        if elementary_inversion_lists_same(target_state_elementary_inversion_set, st_elementary):
                            #print('Its inversion set is already represented in state_list. It has same e-inversion set as ' + st)
                            
                            transition = (generator_word, st)
                            state_info.append(transition)
                            
                            #print('Added the transition pair (' + generator_word + ', ' + st )
                            
                            target_state_is_new = False
                            break
                    
                    if target_state_is_new == True:
                        # add target_state rep to state list
                        state_list.append(target_state_word)
                        #print('New State. Adding to state_list, the rep: ' + word_nice(target_state))
                        num_of_new_states += 1
                                
                        transition = (generator_word, target_state_word)
                        state_info.append(transition)
                        #print('Added the transition pair (' + generator_word + ', ' + target_state_word )

                #else:
                    #print('No, ' + state + ' decreases by ' + word_nice([s(j)]))
                    
            states_transitions.append(state_info)
        
         
        #print('num_of_new_states: ' + str(num_of_new_states))
        #print('NUMBER OF STATES IS NOW: ' + str(len(state_list)))
        #print('State reps are: ')
        #print(state_list)
        #print('The COUNTER is at: ' + str(counter))
        
    print('Number of states: ' + str(len(state_list)))
    print(state_list)
    return states_transitions
            

In [37]:
g2

[['s', ('t', 'ts'), ('u', 'us')],
 ['t', ('s', 'st'), ('u', 'ut')],
 ['u', ('s', 'us'), ('t', 'tu')],
 ['ts', ('s', 'sts'), ('u', 'uts')],
 ['us', ('t', 'tus')],
 ['st', ('t', 'sts'), ('u', 'ust')],
 ['ut', ('s', 'ust'), ('t', 'tut')],
 ['tu', ('s', 'stu'), ('u', 'utu')],
 ['sts', ('u', 'usts')],
 ['uts', ('s', 'usts'), ('t', 'tut')],
 ['tus', ('s', 'stus'), ('u', 'utus')],
 ['ust', ('t', 'tus')],
 ['tut', ('s', 'stu'), ('u', 'utut')],
 ['stu', ('t', 'stus'), ('u', 'ustu')],
 ['utu', ('s', 'ustu'), ('t', 'tutu')],
 ['usts', ('t', 'tusts')],
 ['stus', ('u', 'ustus')],
 ['utus', ('s', 'ustus'), ('t', 'tutu')],
 ['utut', ('s', 'ustu'), ('t', 'tutut')],
 ['ustu', ('t', 'tustu')],
 ['tutu', ('s', 'stutu'), ('u', 'ututu')],
 ['tusts', ('s', 'stus'), ('u', 'utusts')],
 ['ustus', ('t', 'tustus')],
 ['tutut', ('s', 'stutut'), ('u', 'ututut')],
 ['tustu', ('s', 'stustu'), ('u', 'utus')],
 ['stutu', ('t', 'stustu'), ('u', 'ustutu')],
 ['ututu', ('s', 'ustutu'), ('t', 'ututut')],
 ['utusts', ('s',

In [48]:
def Hopcroft(states_transitions):
    '''Applies Hopcroft minimisation to bh-automata'''
    
    # list of states to be emptied
    list_of_states = []
    
    for states in states_transitions:
        list_of_states.append([states[0]])
    
    print('There are ')
    print(len(list_of_states))
    print('number of states')
    
    # copy of list of states inside the list to initiate partition
    partition = [list_of_states[:]]
    print('Partition is: ')
    print(partition)
    
    while len(list_of_states) > 0:
        
        print('Number of states is now: ' + str(len(list_of_states)))
        
        A = list_of_states[0]
        
        #remove from list of states
        list_of_states.remove(A)
        
        print('Set of states removed')
        print(A)
        
        for i in range(0,dim_v):
            generator_word = word_nice([s(i)])
            
            states_with_s_i_transition_to_A = []
            
            for g in A:
                for st in states_transitions:
                    for transition_info in st[1:]:
                        
                        if type(g) == str:

                            if transition_info[0] == generator_word and transition_info[1] == g:
                                states_with_s_i_transition_to_A.append([st[0]])
                                print(st[0] + ' is a state with ' + generator_word + ' transition to ')
                                print(g)
                        
                        elif type(g) == list:
                        
                            if transition_info[0] == generator_word and transition_info[1] == g[0]:
                                states_with_s_i_transition_to_A.append([st[0]])
                                print(st[0] + ' is a state with ' + generator_word + ' transition to ')
                                print(g)
                        
                        else:
                            print('Houston we have a problem')

            print('these are the states with ' + generator_word + ' transitions to ')
            print(g)
            print(states_with_s_i_transition_to_A)
            
            if len(states_with_s_i_transition_to_A) > 0:
                
                #counter = 0
                
                #for y in partition[counter:]:
                for y in partition:
                
                    print('Element y of partition I am looking at: ')
                    print(y)
                    
                    y_and_x = [state for state in y if state in states_with_s_i_transition_to_A]

                    print('y_and_x is ')
                    print(y_and_x)
                    
                    y_minus_x = [i for i in y if i not in states_with_s_i_transition_to_A]

                    print('y_minus_x is ')
                    print(y_minus_x)
                    
                    if y_and_x and y_minus_x:
                        index_of_y = partition.index(y)
                        partition.remove(y)
                        partition.insert(index_of_y, y_and_x)
                        partition.insert(index_of_y+1, y_minus_x)
                        
                        #counter += 2

                        print('Partition is now: ')
                        print(partition)

                        if y in list_of_states:
                            list_of_states.remove(y)
                            list_of_states.append(y_and_x)
                            list_of_states.append(y_minus_x)
                            
                            print('THE LIST OF STATES IS NOW:')
                            print(list_of_states)
                            
                        else: 
                            if len(y_and_x) < len(y_minus_x):
                                list_of_states.append(y_and_x)
                                
                                print('THE LIST OF STATES IS NOW:')
                                print(list_of_states)
                                
                            else:
                                list_of_states.append(y_minus_x)
                                
                                print('THE LIST OF STATES IS NOW:')
                                print(list_of_states)
                        
    return partition

In [49]:
Hopcroft(g2)

There are 
48
number of states
Partition is: 
[[['s'], ['t'], ['u'], ['ts'], ['us'], ['st'], ['ut'], ['tu'], ['sts'], ['uts'], ['tus'], ['ust'], ['tut'], ['stu'], ['utu'], ['usts'], ['stus'], ['utus'], ['utut'], ['ustu'], ['tutu'], ['tusts'], ['ustus'], ['tutut'], ['tustu'], ['stutu'], ['ututu'], ['utusts'], ['tustus'], ['stutut'], ['ututut'], ['stustu'], ['ustutu'], ['ustutut'], ['tustutu'], ['tustutut'], ['utustutu'], ['utustutut'], ['tutustutu'], ['tutustutut'], ['stutustutu'], ['stutustutut'], ['ututustutut'], ['ustutustutut'], ['tustutustutut'], ['utustutustutut'], ['tutustutustutut'], ['stutustutustutut']]]
Number of states is now: 48
Set of states removed
['s']
these are the states with s transitions to 
s
[]
these are the states with t transitions to 
s
[]
these are the states with u transitions to 
s
[]
Number of states is now: 47
Set of states removed
['t']
these are the states with s transitions to 
t
[]
these are the states with t transitions to 
t
[]
these are the states w

y_minus_x is 
[['stus'], ['ustu'], ['tutu'], ['ustus'], ['tutut'], ['stutu'], ['ututu'], ['tustus'], ['stutut'], ['ututut'], ['stustu'], ['ustutu'], ['ustutut'], ['tustutu'], ['tustutut'], ['utustutu'], ['utustutut'], ['tutustutu'], ['tutustutut'], ['stutustutu'], ['stutustutut'], ['ututustutut'], ['ustutustutut'], ['tustutustutut'], ['utustutustutut'], ['tutustutustutut'], ['stutustutustutut']]
Partition is now: 
[[['s']], [['u']], [['t']], [['ts']], [['st']], [['us'], ['ust']], [['ut']], [['uts']], [['tu']], [['tut']], [['sts']], [['tus']], [['tusts']], [['stu']], [['tustu']], [['utu']], [['utut']], [['utus']], [['usts']], [['utusts']], [['stus'], ['ustu'], ['tutu'], ['ustus'], ['tutut'], ['stutu'], ['ututu'], ['tustus'], ['stutut'], ['ututut'], ['stustu'], ['ustutu'], ['ustutut'], ['tustutu'], ['tustutut'], ['utustutu'], ['utustutut'], ['tutustutu'], ['tutustutut'], ['stutustutu'], ['stutustutut'], ['ututustutut'], ['ustutustutut'], ['tustutustutut'], ['utustutustutut'], ['tutustutu

[['tusts']]
Element y of partition I am looking at: 
[['stu']]
y_and_x is 
[]
y_minus_x is 
[['stu']]
Element y of partition I am looking at: 
[['tustu']]
y_and_x is 
[]
y_minus_x is 
[['tustu']]
Element y of partition I am looking at: 
[['utu']]
y_and_x is 
[]
y_minus_x is 
[['utu']]
Element y of partition I am looking at: 
[['utut']]
y_and_x is 
[]
y_minus_x is 
[['utut']]
Element y of partition I am looking at: 
[['utus']]
y_and_x is 
[]
y_minus_x is 
[['utus']]
Element y of partition I am looking at: 
[['usts']]
y_and_x is 
[]
y_minus_x is 
[['usts']]
Element y of partition I am looking at: 
[['utusts']]
y_and_x is 
[]
y_minus_x is 
[['utusts']]
Element y of partition I am looking at: 
[['stus'], ['stustu']]
y_and_x is 
[]
y_minus_x is 
[['stus'], ['stustu']]
Element y of partition I am looking at: 
[['ustu']]
y_and_x is 
[]
y_minus_x is 
[['ustu']]
Element y of partition I am looking at: 
[['tutu']]
y_and_x is 
[]
y_minus_x is 
[['tutu']]
Element y of partition I am looking at: 
[

y_and_x is 
[]
y_minus_x is 
[['ts']]
Element y of partition I am looking at: 
[['st']]
y_and_x is 
[]
y_minus_x is 
[['st']]
Element y of partition I am looking at: 
[['us'], ['ust']]
y_and_x is 
[]
y_minus_x is 
[['us'], ['ust']]
Element y of partition I am looking at: 
[['ut']]
y_and_x is 
[]
y_minus_x is 
[['ut']]
Element y of partition I am looking at: 
[['uts']]
y_and_x is 
[]
y_minus_x is 
[['uts']]
Element y of partition I am looking at: 
[['tu']]
y_and_x is 
[]
y_minus_x is 
[['tu']]
Element y of partition I am looking at: 
[['tut']]
y_and_x is 
[]
y_minus_x is 
[['tut']]
Element y of partition I am looking at: 
[['sts']]
y_and_x is 
[]
y_minus_x is 
[['sts']]
Element y of partition I am looking at: 
[['tus']]
y_and_x is 
[]
y_minus_x is 
[['tus']]
Element y of partition I am looking at: 
[['tusts']]
y_and_x is 
[]
y_minus_x is 
[['tusts']]
Element y of partition I am looking at: 
[['stu']]
y_and_x is 
[]
y_minus_x is 
[['stu']]
Element y of partition I am looking at: 
[['tust

y_minus_x is 
[['ut']]
Element y of partition I am looking at: 
[['uts']]
y_and_x is 
[]
y_minus_x is 
[['uts']]
Element y of partition I am looking at: 
[['tu']]
y_and_x is 
[]
y_minus_x is 
[['tu']]
Element y of partition I am looking at: 
[['tut']]
y_and_x is 
[]
y_minus_x is 
[['tut']]
Element y of partition I am looking at: 
[['sts']]
y_and_x is 
[]
y_minus_x is 
[['sts']]
Element y of partition I am looking at: 
[['tus']]
y_and_x is 
[]
y_minus_x is 
[['tus']]
Element y of partition I am looking at: 
[['tusts']]
y_and_x is 
[]
y_minus_x is 
[['tusts']]
Element y of partition I am looking at: 
[['stu']]
y_and_x is 
[]
y_minus_x is 
[['stu']]
Element y of partition I am looking at: 
[['tustu']]
y_and_x is 
[]
y_minus_x is 
[['tustu']]
Element y of partition I am looking at: 
[['utu']]
y_and_x is 
[]
y_minus_x is 
[['utu']]
Element y of partition I am looking at: 
[['utut']]
y_and_x is 
[]
y_minus_x is 
[['utut']]
Element y of partition I am looking at: 
[['utus']]
y_and_x is 
[]
y_

...

[['ututut']]
y_and_x is 
[]
y_minus_x is 
[['ututut']]
Element y of partition I am looking at: 
[['stutustutustutut']]
y_and_x is 
[]
y_minus_x is 
[['stutustutustutut']]
Element y of partition I am looking at: 
[['ustutu']]
y_and_x is 
[]
y_minus_x is 
[['ustutu']]
Element y of partition I am looking at: 
[['tustutu']]
y_and_x is 
[]
y_minus_x is 
[['tustutu']]
Element y of partition I am looking at: 
[['tustutustutut']]
y_and_x is 
[]
y_minus_x is 
[['tustutustutut']]
Element y of partition I am looking at: 
[['ustutut']]
y_and_x is 
[]
y_minus_x is 
[['ustutut']]
Element y of partition I am looking at: 
[['utustutu']]
y_and_x is 
[]
y_minus_x is 
[['utustutu']]
Element y of partition I am looking at: 
[['utustutustutut']]
y_and_x is 
[]
y_minus_x is 
[['utustutustutut']]
Element y of partition I am looking at: 
[['tustutut']]
y_and_x is 
[]
y_minus_x is 
[['tustutut']]
Element y of partition I am looking at: 
[['utustutut']]
y_and_x is 
[]
y_minus_x is 
[['utustutut']]
Element y of 

[['ututu']]
Element y of partition I am looking at: 
[['ututustutut']]
y_and_x is 
[]
y_minus_x is 
[['ututustutut']]
Element y of partition I am looking at: 
[['tutustutustutut']]
y_and_x is 
[['tutustutustutut']]
y_minus_x is 
[]
Element y of partition I am looking at: 
[['stutu']]
y_and_x is 
[]
y_minus_x is 
[['stutu']]
Element y of partition I am looking at: 
[['stutut']]
y_and_x is 
[]
y_minus_x is 
[['stutut']]
Element y of partition I am looking at: 
[['stutustutu']]
y_and_x is 
[]
y_minus_x is 
[['stutustutu']]
Element y of partition I am looking at: 
[['ututut']]
y_and_x is 
[]
y_minus_x is 
[['ututut']]
Element y of partition I am looking at: 
[['stutustutustutut']]
y_and_x is 
[]
y_minus_x is 
[['stutustutustutut']]
Element y of partition I am looking at: 
[['ustutu']]
y_and_x is 
[]
y_minus_x is 
[['ustutu']]
Element y of partition I am looking at: 
[['tustutu']]
y_and_x is 
[]
y_minus_x is 
[['tustutu']]
Element y of partition I am looking at: 
[['tustutustutut']]
y_and_x

y_and_x is 
[]
y_minus_x is 
[['tu']]
Element y of partition I am looking at: 
[['tut']]
y_and_x is 
[]
y_minus_x is 
[['tut']]
Element y of partition I am looking at: 
[['sts']]
y_and_x is 
[]
y_minus_x is 
[['sts']]
Element y of partition I am looking at: 
[['tus']]
y_and_x is 
[]
y_minus_x is 
[['tus']]
Element y of partition I am looking at: 
[['tusts']]
y_and_x is 
[]
y_minus_x is 
[['tusts']]
Element y of partition I am looking at: 
[['stu']]
y_and_x is 
[]
y_minus_x is 
[['stu']]
Element y of partition I am looking at: 
[['tustu']]
y_and_x is 
[]
y_minus_x is 
[['tustu']]
Element y of partition I am looking at: 
[['utu']]
y_and_x is 
[]
y_minus_x is 
[['utu']]
Element y of partition I am looking at: 
[['utut']]
y_and_x is 
[]
y_minus_x is 
[['utut']]
Element y of partition I am looking at: 
[['utus']]
y_and_x is 
[]
y_minus_x is 
[['utus']]
Element y of partition I am looking at: 
[['usts']]
y_and_x is 
[]
y_minus_x is 
[['usts']]
Element y of partition I am looking at: 
[['utus

y_and_x is 
[]
y_minus_x is 
[['tutut']]
Element y of partition I am looking at: 
[['ututu']]
y_and_x is 
[]
y_minus_x is 
[['ututu']]
Element y of partition I am looking at: 
[['ututustutut']]
y_and_x is 
[]
y_minus_x is 
[['ututustutut']]
Element y of partition I am looking at: 
[['tutustutustutut']]
y_and_x is 
[]
y_minus_x is 
[['tutustutustutut']]
Element y of partition I am looking at: 
[['stutu']]
y_and_x is 
[]
y_minus_x is 
[['stutu']]
Element y of partition I am looking at: 
[['stutut']]
y_and_x is 
[]
y_minus_x is 
[['stutut']]
Element y of partition I am looking at: 
[['stutustutu']]
y_and_x is 
[]
y_minus_x is 
[['stutustutu']]
Element y of partition I am looking at: 
[['ututut']]
y_and_x is 
[]
y_minus_x is 
[['ututut']]
Element y of partition I am looking at: 
[['stutustutustutut']]
y_and_x is 
[]
y_minus_x is 
[['stutustutustutut']]
Element y of partition I am looking at: 
[['ustutu']]
y_and_x is 
[]
y_minus_x is 
[['ustutu']]
Element y of partition I am looking at: 
[[

[[['s']],
 [['u']],
 [['t']],
 [['ts']],
 [['st']],
 [['us'], ['ust']],
 [['ut']],
 [['uts']],
 [['tu']],
 [['tut']],
 [['sts']],
 [['tus']],
 [['tusts']],
 [['stu']],
 [['tustu']],
 [['utu']],
 [['utut']],
 [['utus']],
 [['usts']],
 [['utusts']],
 [['stus'], ['stustu']],
 [['ustu']],
 [['tutu']],
 [['tutustutu']],
 [['tustus']],
 [['ustus']],
 [['tutut']],
 [['ututu']],
 [['ututustutut']],
 [['tutustutustutut']],
 [['stutu']],
 [['stutut']],
 [['stutustutu']],
 [['ututut']],
 [['stutustutustutut']],
 [['ustutu']],
 [['tustutu']],
 [['tustutustutut']],
 [['ustutut']],
 [['utustutu']],
 [['utustutustutut']],
 [['tustutut']],
 [['utustutut']],
 [['tutustutut']],
 [['stutustutut']],
 [['ustutustutut']]]

In [43]:
test = ['sts']
test[0]

'sts'